In [11]:
import re
import os
import pickle
import keras
import statistics

import numpy as np
import pandas as pd

from os import listdir
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence

EAN_REGEX = '(?<=\s)\d{13}(?=\s)'
MODEL_DIR = \
    r'C:\Users\mail\PycharmProjects\MLDM\Data\Final\NAME_NUM_OTHER_Oversampled\models\NAME_NUM_OTHER_OVER_WORDEMB.h5'
TOKENIZER_DIR =\
    r'C:\Users\mail\PycharmProjects\MLDM\Data\Final\NAME_NUM_OTHER_Oversampled\models\NAME_NUM_OTHER_OVER_WORDEMB.pkl'

THRESHOLD = 0.51
SAMPLE_AMOUNT = 100
RANDOM_STATE = 7
TARGETS = ['PROD_NAME', 'PROD_NUM', 'OTHER']
model = keras.models.load_model(MODEL_DIR)
tokenizer = pickle.load(open(TOKENIZER_DIR, 'rb'))
le = LabelEncoder()
print('Model, Tokenizer and LabelEncoder loaded.')

def predictClass(text, tok, model):
    text_pad = sequence.pad_sequences(tok.texts_to_sequences([text]), maxlen=300)
    predict_x = model.predict(text_pad)
    predict_class = np.argmax(predict_x, axis=1)
    score = le.inverse_transform(predict_class)
    prediction = score[0]
    return prediction

PATH = r'C:\Users\mail\Downloads\data\Eval Datasets'
list_files = listdir(PATH)
k = 0
eval_list = []
percent_list = []
percent_list_known = []


def evaluate(num):
    correct = 0
    dataset_filename = os.listdir(PATH)[num]
    print('________________________________________________________________________')
    print('----------{}----------'.format(dataset_filename))
    dataset_path = os.path.join("../..", PATH, dataset_filename)
    df = pd.read_csv(dataset_path, error_bad_lines=False, engine='c',
                 encoding='ISO-8859-14', low_memory=False, dtype=str)
    if len(df.index) >= SAMPLE_AMOUNT:
        try:
            df = df.sample(SAMPLE_AMOUNT,
                           random_state=RANDOM_STATE)
        except ValueError:
            pass
    df = df.reset_index(drop=True)
    column_headers = list(df.columns)
    # list of list of predictions including original column name
    predictions_map = list(map(lambda item: [item], column_headers))

    # list of list of predictions only - populate now then popped later
    predictions_only = list(map(lambda item: [item], column_headers))

    map_list = []
    maps_object = []
    Predictions = []
    Certainty = []
    manual_maps_list = []

    for n in range(len(column_headers)):
        print('----------Mapping column {num} of {len}----------'
              .format(num=n+1, len=len(column_headers)))
        df_select = df[column_headers[n]]
        # sequence padding and tokenization of data requires string type
        df_select = df_select.astype(str)
        multiple_predictions = []
        multiple_certainties = []

        for m in range(len(df_select)):
            class_predictions =[]
            targets = TARGETS
            targets = le.fit_transform(targets)
            data = df_select[m]
            if re.search(EAN_REGEX, data):
                predicted_class = 'PROD_BARCODE_NUM'
            elif len(data) > 200 or np.isnan(data):
                predicted_class = 'OTHER'
            else:
                predicted_class = predictClass(data, tokenizer, model)
            predictions_map[n].append(predicted_class)
            predictions_only[n].append(predicted_class)
            print('Data: {data}, Class Prediction: {prediction}'
                  .format(data=data, prediction=predicted_class))

        predictions_only[n].pop(0)
        column_predictions = (predictions_only[n])
        column_predictions_series = pd.Series(column_predictions)
        column_predictions_count = column_predictions_series.value_counts()
        print('----------Column Mapping Summary----------')
        print('column predictions:', column_predictions)
        #print('column predictions series:', column_predictions_series)
        #print('column predictions count:', column_predictions_count)
        print('predicted class:', column_predictions_count.index[0])
        print('number of classifications:', column_predictions_count.iloc[0])
        print('actual class:', manual_maps[num][n])

        if column_predictions_count.iloc[0] > len(df_select) * THRESHOLD:
            print(column_predictions_count.index[0], 'is the Majority Predicted Class.')
            print('The majority class is: {pred}; {num_pred} of {len} predictions.'
                  '\nOriginal Class: {origin}'
                  .format(pred=column_predictions_count.index[0],
                          num_pred=column_predictions_count.iloc[0],
                          len=len(df_select),
                          origin=column_headers[n]))
            Predictions.append(column_predictions_count.index[0])
            Certainty.append(100)

        else:
            print('There is no Majority Predicted Class above the threshold.'
                  '\nOriginal Class: {origin}'
                  .format(origin=column_headers[n]))
            for i in range(len(column_predictions_count)):
                print('Predicted class {i}: {pred}; {num_pred} of {len} predictions.'
                      .format(i=i+1,
                              pred=column_predictions_count.index[i],
                              num_pred=column_predictions_count.iloc[i],
                              len=len(df_select)))
                multiple_predictions.append(column_predictions_count.index[i])
                multiple_certainties.append(column_predictions_count.iloc[i] / len(df_select) * 100)
            Predictions.append(multiple_predictions)
            Certainty.append(multiple_certainties)
        if column_predictions_count.index[0] == manual_maps[num][n]:
                correct +=1
        print('________________________________________________')

    print('----------Dataset Mapping Summary----------')
    percent = correct/len(column_headers)*100
    print('Correctly mapped columns: {correct}/{total}'.format(correct=correct, total=len(column_headers)))
    print('Correctly mapped column percentage: {percent}'.format(percent=percent))
    percent_list.append(percent)
    print('map list:', predictions_map)
    print('predictions only:', predictions_only)
    for item in range(len(manual_maps[num])):
        temp = manual_maps[num][item]
        manual_maps_list.append(temp)
    print('manual map:', manual_maps_list)
    print('Predictions:', Predictions)
    print('Certainties:', Certainty)





Model, Tokenizer and LabelEncoder loaded.


In [12]:
manual_map_A_dataset = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'PROD_BARCODE_NUM', 5: 'OTHER',
                        6: 'OTHER', 7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12 : 'OTHER',
                        13: 'OTHER', 14: 'OTHER', 15: 'OTHER', 16: 'OTHER', 17: 'OTHER'}
manual_map_A_Discount_bigbuy_da = {0: 'PROD_NUM', 1: 'OTHER', 2: 'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER'}
manual_map_A_Discount_bigbuy_en = {0: 'PROD_NUM', 1: 'OTHER', 2: 'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER'}
manual_map_A_Discount_Compressed1 = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4: 'OTHER',
                                     5: 'OTHER'}
manual_map_A_Discount_Compressed2 = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4:'OTHER'}
manual_map_A_Discount_presta_product_2399_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_2403_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'OTHER'}
manual_map_A_Discount_presta_product_2507_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5:
    'PROD_BARCODE_NUM', 6: 'OTHER', 7: 'OTHER'}
manual_map_A_Discount_presta_product_2507_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM', 6: 'OTHER', 7: 'OTHER'}
manual_map_A_Discount_presta_product_2570_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM'}
manual_map_A_Discount_presta_product_2570_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM'}
manual_map_A_Discount_presta_product_2662_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER'}
manual_map_A_Discount_presta_product_2662_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_2678_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER'}
manual_map_A_Discount_presta_product_2678_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_3046_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'OTHER', 6: 'PROD_BARCODE_NUM', 7: 'OTHER', 8: 'OTHER'}
manual_map_A_Discount_presta_product_3046_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'OTHER', 6: 'PROD_BARCODE_NUM', 7: 'OTHER', 8: 'OTHER'}
manual_map_Bisgaard = {0: 'PROD_BARCODE_NUM', 1: 'OTHER', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER', 7: 'OTHER',
             8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER', 14: 'OTHER', 15: 'OTHER',
            16: 'OTHER', 17: 'OTHER', 18: 'OTHER', 19: 'OTHER', 20: 'OTHER', 21: 'OTHER', 22: 'OTHER', 23: 'OTHER',
            24: 'OTHER'}
manual_map_Scand = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                    7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER'}
manual_map_items = {0: 'PROD_NUM', 1: 'PROD_BARCODE_NUM', 2:'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                    7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER'}
manual_map_Joha = {0: 'OTHER', 1: 'OTHER', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'PROD_BARCODE_NUM', 6: 'OTHER',
                   7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER', 14: 'OTHER',
                   15: 'OTHER', 16: 'OTHER' }
manual_map_Modern_classic_upstart = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'PROD_BARCODE_NUM', 3: 'OTHER', 4: 'OTHER',
                                     5: 'OTHER', 6: 'OTHER', 7: 'OTHER'}
manual_map_PIF = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'PROD_BARCODE_NUM', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                  7: 'OTHER'}
manual_map_prisliste ={0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                       7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER',
                       14:'OTHER', 15: 'OTHER', 16: 'OTHER', 17: 'PROD_BARCODE_NUM', 18: 'OTHER'}
manual_map_VAREFIL = {0: 'PROD_NUM', 1: 'PROD_BARCODE_NUM', 2: 'PROD_NAME', 3: 'PROD_NAME', 4: 'OTHER',  5: 'OTHER',
                      6: 'OTHER'}

manual_maps = [manual_map_A_dataset, manual_map_A_Discount_bigbuy_da, manual_map_A_Discount_bigbuy_en,
               manual_map_A_Discount_Compressed1, manual_map_A_Discount_Compressed2, manual_map_A_Discount_presta_product_2399_da,
               manual_map_A_Discount_presta_product_2403_en, manual_map_A_Discount_presta_product_2507_da,
               manual_map_A_Discount_presta_product_2507_en, manual_map_A_Discount_presta_product_2570_da,
               manual_map_A_Discount_presta_product_2570_en, manual_map_A_Discount_presta_product_2662_da,
               manual_map_A_Discount_presta_product_2662_en, manual_map_A_Discount_presta_product_2678_da,
               manual_map_A_Discount_presta_product_2678_en, manual_map_A_Discount_presta_product_3046_da,
               manual_map_A_Discount_presta_product_3046_en, manual_map_Bisgaard, manual_map_Scand, manual_map_items,
              manual_map_Joha, manual_map_Modern_classic_upstart, manual_map_PIF, manual_map_prisliste, manual_map_VAREFIL]

In [13]:
for i in range(len(list_files)):
    evaluate(i)
#evaluate(0)
#evaluate(1)

________________________________________________________________________
----------A_dataset.csv----------
----------Mapping column 1 of 18----------
Data: 235F6X, Class Prediction: PROD_NUM
Data: 2384TB, Class Prediction: PROD_NUM
Data: 238EB5, Class Prediction: PROD_NUM
Data: AJ6TU2, Class Prediction: PROD_NUM
Data: 234DW8, Class Prediction: PROD_NUM
Data: 2394EG, Class Prediction: PROD_NUM
Data: 235PC7, Class Prediction: PROD_NUM
Data: 237GB4, Class Prediction: PROD_NUM
Data: 235AQ7, Class Prediction: PROD_NUM
Data: AM32GN, Class Prediction: PROD_NUM
Data: 237V58, Class Prediction: PROD_NUM
Data: AG7Z9K, Class Prediction: PROD_NUM
Data: 235T7E, Class Prediction: PROD_NUM
Data: AN637D, Class Prediction: PROD_NUM
Data: AD8V6Q, Class Prediction: PROD_NUM
Data: AK593Y, Class Prediction: PROD_NUM
Data: AR83SE, Class Prediction: PROD_NUM
Data: 235KD7, Class Prediction: PROD_NUM
Data: AM8K3N, Class Prediction: PROD_NUM
Data: AK642J, Class Prediction: PROD_NUM
Data: 237J4K, Class Prediction

KeyboardInterrupt: 

In [ ]:
print('EVAL', eval_list)
print('OVERALL', percent_list)
average_total_autoperf = statistics.mean(percent_list)
print('Average automation percentage:', average_total_autoperf)


